<a href="https://colab.research.google.com/github/anny11020/Colab/blob/main/GestureTrain_resnet50_20201118_PytrochLightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%load_ext tensorboard

In [ ]:
#%tensorboard --logdir lightning_logs

In [ ]:
pip install pytorch-lightning==0.9.0

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torchvision
from torchvision import transforms,models
import pytorch_lightning as pl
import pytorch_lightning.metrics.functional as FM

In [ ]:
img_data = torchvision.datasets.ImageFolder('/content/drive/MyDrive/Colab Notebooks/Gesture/Dataset',
                        transform = transforms.Compose([
                        transforms.Resize(256),
                        transforms.CenterCrop(224),
                        transforms.ToTensor()])
                        )
print(img_data.class_to_idx)

{'both': 0, 'left': 1, 'none': 2, 'right': 3}


In [ ]:
print(len(img_data))

590


In [ ]:
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split

In [ ]:
def train_val_dataset(dataset, val_split=0.25):
  train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split, random_state=7298)
  datasets = {}
  datasets['train'] = Subset(dataset, train_idx)
  datasets['val'] = Subset(dataset, val_idx)
  return datasets

In [ ]:
datasets = train_val_dataset(img_data)
print(len(datasets['train']))
print(len(datasets['val']))

442
148


In [ ]:
# data
train_loader = DataLoader(datasets['train'], batch_size=32, shuffle=True)
val_loader = DataLoader(datasets['val'], batch_size=32, shuffle=False)

In [ ]:
import datetime
today = datetime.date.today()
print(today)

2020-11-18


In [ ]:
class CustomModel(pl.LightningModule):
  def __init__(self, model):
    super().__init__()
    self.model = model

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
    return optimizer

  def training_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self.model(x)
    loss = F.cross_entropy(y_hat, y)
    acc = FM.accuracy(y_hat, y)
    result = pl.TrainResult(minimize=loss)
    result.log('train_loss', loss, prog_bar=True)
    result.log('train_acc', acc, prog_bar=True)
    return result

  def validation_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self.model(x)
    loss = F.cross_entropy(y_hat, y)
    acc = FM.accuracy(y_hat, y)
    result = pl.EvalResult(checkpoint_on=loss,early_stop_on=loss)
    result.log('val_loss', loss, prog_bar=True)
    result.log('val_acc', acc, prog_bar=True)
    return result

  def test_step(self, batch, batch_idx):
    metrics = self.validation_step(batch, batch_idx)
    metrics = {'test_acc': metrics['val_acc'], 'test_loss': metrics['val_loss']}
    self.log_dict(metrics)

In [ ]:
resnet50 = models.resnet50()

In [ ]:
# model
model = CustomModel(resnet50)

In [ ]:
# training
trainer = pl.Trainer(gpus=1,precision=16, limit_train_batches=0.5)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


In [ ]:
trainer.fit(model, train_loader, val_loader)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 25 M  


In [ ]:
torch.save(model,'/content/drive/MyDrive/Colab Notebooks/Gesture/Train1118/model_'+str(today)+'.pth')